<a href="https://colab.research.google.com/github/cookie-pan/Python-Data-Visualization/blob/main/project/project_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
Project for Week 4 of "Python Data Visualization".
Unify data via common country codes.

Be sure to read the project description page for further information
about the expected behavior of the program.
author:Pan_xin
"""

import csv
import math
import pygal


def build_country_code_converter(codeinfo):
    """
    Inputs:
      codeinfo      - A country code information dictionary

    Output:
      A dictionary whose keys are plot country codes and values
      are world bank country codes, where the code fields in the
      code file are specified in codeinfo.
    """

    plot_country={}

    with open(codeinfo["codefile"],"r") as cfile:
        total=csv.DictReader(cfile,delimiter=codeinfo["separator"],quotechar=codeinfo["quote"])

        for item in total:
            row_data=dict(item)
            plot_country[row_data[codeinfo["plot_codes"]]]=row_data[codeinfo["data_codes"]]

    return plot_country



def reconcile_countries_by_code(codeinfo, plot_countries, gdp_countries):
    """
    Inputs:
      codeinfo       - A country code information dictionary
      plot_countries - Dictionary whose keys are plot library country codes
                       and values are the corresponding country name
      gdp_countries  - Dictionary whose keys are country codes used in GDP data

    Output:
      A tuple containing a dictionary and a set.  The dictionary maps
      country codes from plot_countries to country codes from
      gdp_countries.  The set contains the country codes from
      plot_countries that did not have a country with a corresponding
      code in gdp_countries.

      Note that all codes should be compared in a case-insensitive
      way.  However, the returned dictionary and set should include
      the codes with the exact same case as they have in
      plot_countries and gdp_countries.
    """
    found_country_codes = {}
    no_code_countries = set()

    plotcode_code=build_country_code_converter(codeinfo)
    ang=list(gdp_countries.keys())
    cng= [x.lower() for x in ang]

    bng=list(plotcode_code.keys())
    dng= [x.lower() for x in bng]

    for item in plot_countries:
        if item.lower() in dng:
            gdp_code=plotcode_code[bng[dng.index(item.lower())]]   #返回gdp的code 

            if gdp_code.lower() in cng:  
                found_country_codes[item] = ang[cng.index(gdp_code.lower())] 
          
            else:
                no_code_countries.add(item) 

        else:
            no_code_countries.add(item) #改

    return_tuple=()+(found_country_codes,no_code_countries,)

    return return_tuple


def read_csv_as_nested_dict(filename, keyfield, separator, quote):
    """
    Inputs:
      filename  - Name of CSV file
      keyfield  - Field to use as key for rows
      separator - Character that separates fields
      quote     - Character used to optionally quote fields

    Output:
      Returns a dictionary of dictionaries where the outer dictionary
      maps the value in the key_field to the corresponding row in the
      CSV file.  The inner dictionaries map the field names to the
      field values for that row.
    """
    with open (filename,"r") as fpg:
        total_data=csv.DictReader(fpg,delimiter=separator, quotechar=quote)
        total_dict={}

        for item in total_data:
            single_dict=dict(item)
            total_dict[single_dict.get(keyfield)]=single_dict
          
    return total_dict




def build_map_dict_by_code(gdpinfo, codeinfo, plot_countries, year):
    """
    Inputs:
      gdpinfo        - A GDP information dictionary
      codeinfo       - A country code information dictionary
      plot_countries - Dictionary mapping plot library country codes to country names
      year           - String year for which to create GDP mapping

    Output:
      A tuple containing a dictionary and two sets.  The dictionary
      maps country codes from plot_countries to the log (base 10) of
      the GDP value for that country in the specified year.  The first
      set contains the country codes from plot_countries that were not
      found in the GDP data file.  The second set contains the country
      codes from plot_countries that were found in the GDP data file, but
      have no GDP data for the specified year.
    """
    code_gdp={}
    unex_country=set()
    unex_year=set()

    total_dict=read_csv_as_nested_dict(gdpinfo["gdpfile"],
                            gdpinfo["country_code"], gdpinfo["separator"], gdpinfo["quote"])
    data=reconcile_countries_by_code(codeinfo, plot_countries, total_dict)
    unex_country=data[1]
    plotcode_code=data[0] #存在gdp国家代码的地图代码

    for items in plotcode_code:
        try:
            code_gdp[items]=math.log10(float(total_dict[plotcode_code[items]][year])) 

        except ValueError:
            unex_year.add(items)

    returned_tuple=(code_gdp,unex_country,unex_year,)

    return returned_tuple


def render_world_map(gdpinfo, codeinfo, plot_countries, year, map_file):
    """
    Inputs:
      gdpinfo        - A GDP information dictionary
      codeinfo       - A country code information dictionary
      plot_countries - Dictionary mapping plot library country codes to country names
      year           - String year of data
      map_file       - String that is the output map file name

    Output:
      Returns None.

    Action:
      Creates a world map plot of the GDP data in gdp_mapping and outputs
      it to a file named by svg_filename.
    """

    data=build_map_dict_by_code(gdpinfo, codeinfo, plot_countries, year)
    code_gdp=data[0]
    unex_country=data[1]
    unex_year=data[2]

    worldmap_chart = pygal.maps.world.World()
    worldmap_chart.title = 'GDP by country for ' + year + \
                    ' (log scale), unified by common' + ' country NAME'

    worldmap_chart.add('GDP For ' + year, code_gdp)
    worldmap_chart.add('Missing from World Bank Data',list(unex_country))
    worldmap_chart.add('No GDP Data', list(unex_year))

    worldmap_chart.render_to_file(map_file)